In [16]:
# Basic constants

OUTPUT_DIR = '/home/ubuntu/visual_search_assistant/M3/results'
FACE_LIB_DIR = '/home/ubuntu/visual_search_assistant/M3/library/'
DATA_DIR = '/home/ubuntu/visual_search_assistant/data'
SAMPLE_FRAME_FREQ = 2
SAMPLE_CLUSTER_FREQ = 20
LOG_FREQ = 50

batch_size = 128

input_video = None
output_video = None


## Download Test Videos

Functions to download youtube video from link and clip them 

In [79]:
from pytube import YouTube 
from moviepy.editor import *

def download_video(link,output_path,output_title=None):
    yt = YouTube(link)
    video_stream = yt.streams.first()
    video_stream.download(output_path=output_path,filename=output_title)
    print('Download complete for %s' % video_stream.title)
    return video_stream.title


def clip_video(video_pth,output_pth,time):
    """
    video_pth: /path/to/input/video
    output_pth: /path/to/output/video
    time: list of start and end time in seconds to clip video
    """

    start,end=time
    clip = VideoFileClip(video_pth).subclip(start,end)
    if output_pth is None:
        name = video_pth.split('/')[-1].split('.')[0]
        new_name = name + '_%d_%d'%(start,end)
        output_pth = video_pth.replace(name,new_name)
        
    #import pdb; pdb.set_trace()
    clip.write_videofile(output_pth)
        

Specify the link and download

In [72]:
download_link = 'https://www.youtube.com/watch?v=oHddkxW5IqU'
download_link = 'https://www.youtube.com/watch?v=X4uo7uKR61M'
fname = 'radio_star_.mp4'
fname = 'parks_and_rec_.mp4'

output_pth = os.path.join(DATA_DIR,fname)
if not os.path.exists(output_pth):
    download_video(download_link,DATA_DIR,fname)

Download complete for Donna Wakes Up Everyday And Chooses Being Cool | Parks and Recreation


In [75]:
from IPython.core.display import Video
Video('/home/ubuntu/visual_search_assistant/data/Donna Wakes Up Everyday And Chooses Being Cool  Parks and Recreation.mp4')

## Library of face embeddings

In [2]:
import os
import face_recognition

faces = os.listdir(FACE_LIB_DIR)
all_embeddings = []
all_names = []
for file in faces:
    face = face_recognition.load_image_file(os.path.join(FACE_LIB_DIR,file))
    all_embeddings.append(face_recognition.face_encodings(face)[0])
    name = file.split('/')[-1].split('.')[0]
    all_names.append(name)
    

In [39]:
import os
import cv2
import pickle
import numpy as np


def process_video(input_pth,output_pth=None,use_gpu=True,recognition=False,detection_threshold=0.7,
                  result_dir=None,max_frames=None,sample_cluster_freq=2,sample_freq=2,batch_size=1):
    if output_pth is None:
        output_pth = os.path.join(OUTPUT_DIR,input_pth.split('/')[-1])
    if use_gpu:
        batch_size = 16
    
    if result_dir is None:
        import pdb;pdb.set_trace()
        input_name = input_pth.split('/')[-1].split('.')[0]
        result_dir = os.path.join(OUTPUT_DIR,input_name)
    
    if not os.path.exists(result_dir):
        sampled_face_dir = os.path.join(result_dir,'sampled_faces')
        os.makedirs(sampled_face_dir)
    else:
        print('Error result dir %s already exists! aborting' % result_dir)
        sampled_face_dir = os.path.join(result_dir,'sampled_faces')
#         return
    
    
    cluster_dict = {}
    
    video_capture = cv2.VideoCapture(input_pth)

    frame_width = int(video_capture.get(3))
    frame_height = int(video_capture.get(4))
    
    fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
    out = cv2.VideoWriter(output_pth, fourcc, 30.0, (frame_width, frame_height))
    
    frame_count = 0
    
    frames = []
    print('='*20,'Start Face Detection and Recognition','='*20)
    while video_capture.isOpened():
        ret, frame = video_capture.read()
        
        if not ret or (max_frames and frame_count > max_frames):
            break
        if frame_count % LOG_FREQ == LOG_FREQ -1:
            print('Processed %d frames'% frame_count)
            
        frame_count += 1
        
        # skip frames
        if frame_count % sample_freq > 0:
            continue
            
        frames.append(frame)
        if len(frames) == batch_size:
#             import pdb;pdb.set_trace()
            batch_face_locations = face_recognition.batch_face_locations(frames)
            for idx,face_locations in enumerate(batch_face_locations):
                number_of_faces_in_frame = len(face_locations)
                
                fno = frame_count - batch_size + idx
                frame = frames[idx]
                
                
                embeddings = face_recognition.face_encodings(frame,face_locations)
                for iidx,(embd,(top,right,bottom,left)) in enumerate(zip(embeddings,face_locations)):
                    # Draw a box around the face
                    cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
                    # Draw a label with a name below the face
                    cv2.rectangle(frame, (left, bottom + 35), (right, bottom), (0, 0, 255), cv2.FILLED)
                    
                    
                    # write label
                    face_dist = face_recognition.face_distance(all_embeddings,embd)
                    i = np.argmin(face_dist)
                    name = all_names[i] if face_dist[i] > detection_threshold else 'Unknown'
                    font = cv2.FONT_HERSHEY_DUPLEX
                    cv2.putText(frame, name, (left + 6, bottom + 6), font, 0.5, (255, 255, 255), 1)
                    
                    
                    # save face image for clustering
                    if idx % sample_cluster_freq ==0:
                        fname = '%.3d_%.3d.png'%(fno,iidx)
                        face_img = frame[top:bottom,left:right,:]
                        # save face image
                        cv2.imwrite(os.path.join(sampled_face_dir,fname),face_img)
                        # save embd in dict
                        cluster_dict[fname.split('.')[0]] = embd
                                
                out.write(frame)
            frames = []
                
                
    video_capture.release()
    out.release()
    
    # save sampled embeddings as pickle file
    import pdb;pdb.set_trace()
    cluster_meta_file = os.path.join(result_dir,'embeddings.pickle')
    with open(cluster_meta_file,'wb') as f:
        pickle.dump(cluster_dict,f,protocol=pickle.HIGHEST_PROTOCOL)
    
    print('='*15,'Done Processing %s to %s' % (input_pth,output_pth),'='*15)
        

In [40]:
test_input_video = '/home/ubuntu/visual_search_assistant/data/radio_star_10_20.mp4'
process_video(test_input_video,max_frames=64,batch_size=16)

> <ipython-input-39-aa42399c9849>(16)process_video()
-> input_name = input_pth.split('/')[-1].split('.')[0]
(Pdb) c
Error result dir /home/ubuntu/visual_search_assistant/M3/results/radio_star_10_20 already exists! aborting
==================== Start Face Detection and Recognition ====================
Processed 49 frames
> <ipython-input-39-aa42399c9849>(101)process_video()
-> cluster_meta_file = os.path.join(result_dir,'embeddings.pickle')
(Pdb) cluster_dict.keys()
dict_keys(['016_000', '016_001', '018_000', '018_001', '020_000', '020_001', '022_000', '022_001', '024_000', '024_001', '026_000', '026_001', '028_000', '028_001', '030_000', '030_001', '048_000', '048_001', '048_002', '050_000', '050_001', '052_000', '052_001', '054_000', '054_001', '056_000', '056_001', '058_000', '060_000', '062_000', '062_001'])
(Pdb) c
=============== Done Processing /home/ubuntu/visual_search_assistant/data/radio_star_10_20.mp4 to /home/ubuntu/visual_search_assistant/M3/results/radio_star_10_20.mp4 ==

### Check sampled faces & embeddings

In [53]:
embd_pkl_file = '/home/ubuntu/visual_search_assistant/M3/results/radio_star_10_20/embeddings.pickle'

with open(embd_pkl_file,'rb') as rf:
    embedding_dict = pickle.load(rf)

embeddings = np.array(list((embedding_dict.values())))

## Cluster Some Faces!
Use a clustering algorithm from Scikit
https://pyimagesearch.com/2018/07/09/face-clustering-with-python/

In [54]:
from sklearn.cluster import DBSCAN

clt = DBSCAN(metric="euclidean", n_jobs=-1)
clt.fit(embeddings)

DBSCAN(n_jobs=-1)

In [58]:
# determine the total number of unique faces found in the dataset
labelIDs = np.unique(clt.labels_)
numUniqueFaces = len(np.where(labelIDs > -1)[0])
print("[INFO] # unique faces: {}".format(numUniqueFaces))

[INFO] # unique faces: 2


array([-1,  1, -1, -1,  0,  1,  0,  1,  0,  1,  0,  1,  0,  1,  0,  1,  0,
        1,  1,  0,  1,  0,  1,  0,  1,  0,  1,  0,  0,  0,  1])